In [50]:
from pymatgen.core.structure import Structure
import numpy as np
import plotly.graph_objects as go
import plotly.io as pio

In [51]:
CIF_PATH = '/mnt/ssd_elecom_black/cif/1/00/00/1000000.cif'

In [24]:
pio.renderers.default='browser'

##1) inport cif file
aspirin = Structure.from_file(CIF_PATH)
                        
aspirin_lattice = aspirin.lattice.matrix
aspirin_atom_num = np.size(aspirin.atomic_numbers)
aspirin_atom_posi = aspirin.cart_coords
aspirin_atom_kind = aspirin.atomic_numbers


# print(f"aspirin_atom_posi[0]:{aspirin_atom_posi[0]}")
# print(f"aspirin_atom_kind:{aspirin_atom_kind}")


aspirin_atom_num:140
aspirin_atom_posi[0]:[ 1.9398984  10.04260639 15.41202798]
aspirin_atom_kind:(13, 13, 13, 13, 15, 15, 15, 15, 15, 15, 15, 15, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8)


In [35]:
print(f"aspirin_atom_num: {aspirin_atom_num}")

aspirin_atom_num: 140


In [25]:
aspirin_atom_posi.shape

(140, 3)

In [29]:
aspirin_atom_posi[0:3]

array([[ 1.9398984 , 10.04260639, 15.41202798],
       [ 1.9833682 ,  4.80815639,  8.33658566],
       [ 5.90663479,  0.42629361, -0.0508007 ]])

In [44]:
aspirin_atom_kind_unique = set(aspirin_atom_kind)
aspirin_atom_kind_unique

{1, 6, 7, 8, 13, 15}

In [53]:
# from pymatgen.core.structure import Structure
# import numpy as np
# import plotly.graph_objects as go
# import plotly.io as pio
# pio.renderers.default='browser'

##1) inport cif file
aspirin = Structure.from_file(CIF_PATH)
                        
aspirin_lattice = aspirin.lattice.matrix
aspirin_atom_num = np.size(aspirin.atomic_numbers)
aspirin_atom_posi = aspirin.cart_coords
aspirin_atom_kind = aspirin.atomic_numbers

a_axis = [[0.0, aspirin_lattice[0][0]], [0.0, aspirin_lattice[0][1]], [0.0, aspirin_lattice[0][2]]]
b_axis = [[0.0, aspirin_lattice[1][0]], [0.0, aspirin_lattice[1][1]], [0.0, aspirin_lattice[1][2]]]
c_axis = [[0.0, aspirin_lattice[2][0]], [0.0, aspirin_lattice[2][1]], [0.0, aspirin_lattice[2][2]]]

##2) make bond list

#covalent radius[AA]
#reference... https://www.hulinks.co.jp/support/c-maker/qa_005.html
covalent_radius_list = [0.00, 0.37, 0.32,
                        1.34, 0.90, 0.82, 0.77, 0.75, 0.73, 0.71, 0.69]

dstance = []
bond_list = []
bond_length = []

for j in range(aspirin_atom_num):
    j_cov_rad = covalent_radius_list[aspirin_atom_kind[j]]
    
    for k in range(aspirin_atom_num):
        if j < k:
            distance = np.linalg.norm(aspirin_atom_posi[j] - aspirin_atom_posi[k])
            
            k_cov_rad = covalent_radius_list[aspirin_atom_kind[k]]
                
            if distance <= j_cov_rad + k_cov_rad:
                bond_list.append([j, k])
                bond_length.append([distance, j_cov_rad + k_cov_rad])
            else:
                pass
            
        else:
            pass

##3) 3D plot by plotly

colorscale = ["powderblue", "black", "black", "black", "green", "lightgrey", "blue", "red"]

trace = go.Scatter3d(
   x = aspirin_atom_posi[:, 0],
   y = aspirin_atom_posi[:, 1],
   z = aspirin_atom_posi[:, 2],
   name = "atoms",
   mode = 'markers', 
   marker = dict(
       size = 7,
       color = aspirin_atom_kind,
       colorscale = ["powderblue", "black", "black", "black", "green", "lightgrey", "blue", "red"]
       )
   )

a_axisp = go.Scatter3d(
   x = a_axis[0], y = a_axis[1], z = a_axis[2],
   name = 'a_axis',
   mode = 'lines',
    marker=dict(size=0, color='red', opacity=0.8)
    )

b_axisp = go.Scatter3d(
   x = b_axis[0], y = b_axis[1], z = b_axis[2],
   name = 'b_axis',
   mode = 'lines',
    marker=dict(size=0, color='green', opacity=0.8)
   )

c_axisp = go.Scatter3d(
   x = c_axis[0], y = c_axis[1], z = c_axis[2],
   name = 'c_axis',
   mode = 'lines',
    marker=dict(size=0, color='blue', opacity=0.8)
  )

axis = [a_axisp] + [b_axisp] + [c_axisp]
bond = []

for l in range(len(bond_list)):
    bond_x = [aspirin_atom_posi[bond_list[l][0]][0], aspirin_atom_posi[bond_list[l][1]][0]]
    bond_y = [aspirin_atom_posi[bond_list[l][0]][1], aspirin_atom_posi[bond_list[l][1]][1]]
    bond_z = [aspirin_atom_posi[bond_list[l][0]][2], aspirin_atom_posi[bond_list[l][1]][2]]
    
    bond_i = go.Scatter3d(
        x = bond_x, y = bond_y, z = bond_z,
        name = 'bond',
        mode = 'lines',
        marker=dict(size=5,line=dict(width=15,), color='gray', opacity=0.8)
        )
    
    bond = bond + [bond_i]
    
fig = go.Figure(data = bond + [trace] + axis)

noaxis=dict(
    showbackground=False,
    showgrid=False,
    showline=False,
    showticklabels=False,
    ticks='',
    title='',        
    zeroline=False)

fig.update_layout( 
    width=1500, height=500,
    margin=dict(l=0, r=0, b=0, t=0),
    scene_aspectmode='data',
    scene = dict(
        xaxis = noaxis,
        yaxis = noaxis,
        zaxis = noaxis),
    plot_bgcolor="gray",
    )

fig.show()



IndexError: list index out of range